1- Importamos librerías a utilizar

In [114]:
import pandas as pd
import xgboost as xgb
import urllib.request
from numpy import array
from sklearn.model_selection import train_test_split
from urllib.request import urlretrieve
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

2- Descargamos los archivos a utilizar

In [38]:
#Guardamos las URLs a utilizar, que contienen la raw data, a objetos
url_data = "https://raw.githubusercontent.com/jboscomendoza/xgboost_en_python/master/agaricus-lepiota.data"
url_names = "https://raw.githubusercontent.com/jboscomendoza/xgboost_en_python/master/agaricus-lepiota.names"


In [39]:
#Y Aquí con urlretireve obtenemos el contenido de las urls
urlretrieve(url_data, "agaricus-lepiota.data")
urlretrieve(url_names, "agaricus-lepiota.names")

('agaricus-lepiota.names', <http.client.HTTPMessage at 0x7b855c67a1a0>)

3- Exploración de datos: abrirlos y asignarlos a una estructuras tabular

In [47]:
#Aquí definimos una función para abrir las primeras 10 líneas de datos y conocer su estructura
def ver_contenido(ruta, mode="r", lineas=10):
    with open(ruta, mode) as archivo:
        contenido = archivo.readlines()
        [print(i.strip()) for i in contenido[:lineas]]


In [48]:
#Vemos que posee estructura tabular
ver_contenido("agaricus-lepiota.data")

p,x,s,n,t,p,f,c,n,k,e,e,s,s,w,w,p,w,o,p,k,s,u
e,x,s,y,t,a,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,g
e,b,s,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m
p,x,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,s,u
e,x,s,g,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g
e,x,y,y,t,a,f,c,b,n,e,c,s,s,w,w,p,w,o,p,k,n,g
e,b,s,w,t,a,f,c,b,g,e,c,s,s,w,w,p,w,o,p,k,n,m
e,b,y,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,s,m
p,x,y,w,t,p,f,c,n,p,e,e,s,s,w,w,p,w,o,p,k,v,g
e,b,s,y,t,a,f,c,b,g,e,c,s,s,w,w,p,w,o,p,k,s,m


In [49]:
ver_contenido("agaricus-lepiota.names")

1. Title: Mushroom Database

2. Sources:
(a) Mushroom records drawn from The Audubon Society Field Guide to North
American Mushrooms (1981). G. H. Lincoff (Pres.), New York: Alfred
A. Knopf
(b) Donor: Jeff Schlimmer (Jeffrey.Schlimmer@a.gp.cs.cmu.edu)
(c) Date: 27 April 1987

3. Past Usage:


In [50]:
#Como es una cadena de texto, no tenemos el nombre de las columnas. Como esa información la tenemos disponible,
#creamos una lista con sus nombres para asignarla luego a un DataFrame
nombres = [
 "target", "cap_shape", "cap_surface", "cap_color", "bruises", "odor",
 "gill_attachment", "gill_spacing", "gill_size", "gill_color", "stalk_shape",
 "stalk_root", "stalk_surface_above_ring", "stalk_surface_below_ring",
 "stalk_color_above_ring", "stalk_color_below_ring", "veil_type",
 "veil_color", "ring_number", "ring_type", "spore_print_color", "population",
 "habitat"
]

In [51]:
#Creamos el df llamar agar
agar = pd.read_csv("agaricus-lepiota.data", names=nombres)

In [52]:
#Lo exploramos para tener una primera visión
agar.head(10)

,target,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
5,e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g
6,e,b,s,w,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,n,m
7,e,b,y,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,s,m
8,p,x,y,w,t,p,f,c,n,p,...,s,w,w,p,w,o,p,k,v,g
9,e,b,s,y,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,s,m


In [57]:
#Miramos que no poseaa nulos
agar.isna().sum()

target                      0
cap_shape                   0
cap_surface                 0
cap_color                   0
bruises                     0
odor                        0
gill_attachment             0
gill_spacing                0
gill_size                   0
gill_color                  0
stalk_shape                 0
stalk_root                  0
stalk_surface_above_ring    0
stalk_surface_below_ring    0
stalk_color_above_ring      0
stalk_color_below_ring      0
veil_type                   0
veil_color                  0
ring_number                 0
ring_type                   0
spore_print_color           0
population                  0
habitat                     0
dtype: int64

In [60]:
agar.dtypes

target                      object
cap_shape                   object
cap_surface                 object
cap_color                   object
bruises                     object
odor                        object
gill_attachment             object
gill_spacing                object
gill_size                   object
gill_color                  object
stalk_shape                 object
stalk_root                  object
stalk_surface_above_ring    object
stalk_surface_below_ring    object
stalk_color_above_ring      object
stalk_color_below_ring      object
veil_type                   object
veil_color                  object
ring_number                 object
ring_type                   object
spore_print_color           object
population                  object
habitat                     object
dtype: object

In [76]:
#Necesitamos tanformar el tipo de valor de las variables, ya que XGBoost requiere matrices
#numéricas para funcionar correctamente

def str_a_num(df):
    for col in df:
        original = df[col].unique()
        reemplazo = list(range(len(original)))
        mapa = dict(zip(original, reemplazo))
        df[col] = df[col].replace(mapa)
    return df

In [77]:
#vemos el resultado
agar = str_a_num(agar)
agar.head(5)


,target,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,1,1
2,1,1,0,2,0,2,0,0,1,1,...,0,0,0,0,0,0,0,1,1,2
3,0,0,1,2,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,3,1,3,0,1,1,0,...,0,0,0,0,0,0,1,1,2,1


4. Generamos el modelo en sí para clasificar hongos venenosos

In [78]:
#ahora generamos el split para luego generar el modelo de ML

agar_train, agar_test = train_test_split(agar, test_size=.3, random_state=1999)

In [79]:
agar_train.shape

(5686, 23)

In [80]:
agar_test.shape

(2438, 23)

In [82]:
agar_train_mat = xgb.DMatrix(agar_train.drop("target", 1), label=agar_train["target"])

<ipython-input-82-e623a9341813>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  agar_train_mat = xgb.DMatrix(agar_train.drop("target", 1), label=agar_train["target"])


In [83]:
agar_test_mat = xgb.DMatrix(agar_test.drop("target", 1), label=agar_test["target"])

<ipython-input-83-016cdf8f2b0a>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  agar_test_mat = xgb.DMatrix(agar_test.drop("target", 1), label=agar_test["target"])


In [84]:
agar_train_mat

In [86]:
#Definimos los parametros del modelo
parametros = {"booster":"gbtree", "max_depth": 2, "eta": 0.3, "objective": "binary:logistic",
"nthread":2}

rondas = 10

In [87]:
evaluacion = [(agar_test_mat, "eval"), (agar_train_mat, "train")]

In [88]:
#Y generamos el modelo
modelo = xgb.train(parametros, agar_train_mat, rondas, evaluacion)

[0]	eval-logloss:0.45257	train-logloss:0.45154
[1]	eval-logloss:0.32120	train-logloss:0.31939
[2]	eval-logloss:0.23969	train-logloss:0.23723
[3]	eval-logloss:0.18634	train-logloss:0.18330
[4]	eval-logloss:0.15047	train-logloss:0.14692
[5]	eval-logloss:0.12284	train-logloss:0.11964
[6]	eval-logloss:0.10287	train-logloss:0.09990
[7]	eval-logloss:0.08615	train-logloss:0.08230
[8]	eval-logloss:0.07460	train-logloss:0.07097
[9]	eval-logloss:0.06607	train-logloss:0.06264


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:727: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


In [89]:
modelo

In [90]:
#Generamos predicciones con nuestro modelo

prediccion = modelo.predict(agar_test_mat)

#Nuestro resultado es un arreglo de valores numéricos, cada uno representando la probabilidad
#de que un caso en particular pertenezca al valor 1 de nuestra variable objetivo.

In [92]:
prediccion

array([0.02315998, 0.9644533 , 0.8598643 , ..., 0.02315998, 0.9644533 ,
       0.9644533 ], dtype=float32)

In [94]:
#Ahora lo que haremos es quedarnos sólo con las predicciones mayores a 0.5 ya que nos dan mayor confianza

prediccion = [1 if i > .5 else 0 for i in prediccion]

prediccion[:10]

[0, 1, 1, 0, 0, 1, 1, 1, 1, 0]

5- Evaluamos el modelo que generamos

In [112]:
#Para evaluar el modelo utilizamos 3 funciones de Scikit Learn confusion_matrix(), accuracy_score(), classification_report()
#Para facilitar el uso de estas funciones, definimos una función que nos devuelve resultados de las tres

def metricas(objetivo, prediccion):
    matriz_conf = confusion_matrix(objetivo, prediccion)
    score = accuracy_score(objetivo, prediccion)
    reporte = classification_report(objetivo, prediccion)
    metricas = [matriz_conf, score, reporte]
    return metricas

In [115]:
metricas = metricas(agar_test["target"], prediccion)
[print(i) for i in metricas]

[[1136   39]
 [   0 1263]]
0.9840032813781788
              precision    recall  f1-score   support

           0       1.00      0.97      0.98      1175
           1       0.97      1.00      0.98      1263

    accuracy                           0.98      2438
   macro avg       0.99      0.98      0.98      2438
weighted avg       0.98      0.98      0.98      2438



[None, None, None]

In [ ]:
#Vemos que los resultados son muy positivo, tal como ofrece el modelo de XGBoost
#Sin embargo, generaremos un segundo modelo para comparar donde ajustaremos las métricas



In [122]:
parametros_02 = {"booster":"gbtree", "max_depth": 4, "eta": .3, "objective": "binary:logistic",
"nthread":2}
rondas_02 = 100
modelo_02 = xgb.train(parametros_02, agar_test_mat, rondas_02, evaluacion,
early_stopping_rounds=10)



[0]	eval-logloss:0.44376	train-logloss:0.44369
[1]	eval-logloss:0.30598	train-logloss:0.30602


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:727: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


[2]	eval-logloss:0.21895	train-logloss:0.21877
[3]	eval-logloss:0.15852	train-logloss:0.15895
[4]	eval-logloss:0.11565	train-logloss:0.11624
[5]	eval-logloss:0.08619	train-logloss:0.08661
[6]	eval-logloss:0.06434	train-logloss:0.06473
[7]	eval-logloss:0.04867	train-logloss:0.04923
[8]	eval-logloss:0.03649	train-logloss:0.03700
[9]	eval-logloss:0.02786	train-logloss:0.02844
[10]	eval-logloss:0.02117	train-logloss:0.02168
[11]	eval-logloss:0.01647	train-logloss:0.01692
[12]	eval-logloss:0.01294	train-logloss:0.01341
[13]	eval-logloss:0.01042	train-logloss:0.01086
[14]	eval-logloss:0.00859	train-logloss:0.00905
[15]	eval-logloss:0.00723	train-logloss:0.00769
[16]	eval-logloss:0.00599	train-logloss:0.00636
[17]	eval-logloss:0.00508	train-logloss:0.00541
[18]	eval-logloss:0.00446	train-logloss:0.00479
[19]	eval-logloss:0.00399	train-logloss:0.00430
[20]	eval-logloss:0.00351	train-logloss:0.00376
[21]	eval-logloss:0.00324	train-logloss:0.00345
[22]	eval-logloss:0.00295	train-logloss:0.00317


In [125]:
#ahora veamos las metricas de evaluación

def metricas2(objetivo, prediccion):
    matriz_conf = confusion_matrix(objetivo, prediccion)
    score = accuracy_score(objetivo, prediccion)
    reporte = classification_report(objetivo, prediccion)
    metricas = [matriz_conf, score, reporte]
    return metricas


prediccion_02 = modelo_02.predict(agar_test_mat)
prediccion_02 = [1 if i > .5 else 0 for i in prediccion_02]
metricas_02 = metricas2(agar_test["target"], prediccion_02)
[print(i) for i in metricas_02]


[[1175    0]
 [   0 1263]]
1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1175
           1       1.00      1.00      1.00      1263

    accuracy                           1.00      2438
   macro avg       1.00      1.00      1.00      2438
weighted avg       1.00      1.00      1.00      2438



[None, None, None]

In [126]:
#Aquí vemos que la precisión del modelo llega al 100%, tanto para la clase positiva como para la
#negativa